## Создание RAG-ассистента на Yandex AI Assistant API

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Yandex AI Assistant API с поддержкой RAG.

Для начала, установим Yandex Cloud ML SDK:

In [ ]:
%pip install --upgrade --quiet yandex-cloud-ml-sdk

Также необходимо обновить некоторые библиотеки:

In [ ]:
%pip install --upgrade --quiet pydantic

**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

И ещё пара полезных функций на будущее:

In [1]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

Для работы с языковыми моделями нам понадобится ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere.

Создадим модель последней версии YandexGPT 5 и убедимся, что она кое-что знает про вина:

In [2]:
import os
from yandex_cloud_ml_sdk import YCloudML

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

sdk = YCloudML(folder_id=folder_id, auth=api_key)

# Раскомментируйте, если хотите подробнее смотреть, что делает SDK
#sdk.setup_default_logging(log_level='DEBUG')

model = sdk.models.completions("yandexgpt", model_version="rc")

In [3]:
printx(model.run("Какое вино можно пить со стейком?").text)

Выбор вина к стейку зависит от степени прожарки мяса, его вида и ваших личных предпочтений. Вот несколько примеров вин, которые могут хорошо сочетаться со стейком:

1. **Каберне Совиньон** — это насыщенный и крепкий сорт вина с нотами тёмного ягодного вкуса и танинов, который может хорошо дополнить вкус жареного мяса.

2. **Мерло** — более мягкий и фруктовый сорт вина, который также может хорошо сочетаться со стейком.

3. **Шираз (Сира)** — это насыщенное и пряное вино с нотами тёмной вишни, ежевики и специй, которое может хорошо подойти к стейку.

4. **Мальбек** — аргентинский сорт вина с насыщенным вкусом и ароматом тёмных ягод, который также может быть хорошим выбором для стейка.

5. **Темпранильо** — испанское вино с нотами красных ягод, ванили и специй, которое может хорошо сочетаться с мясом.

6. **Бордо** — французское вино из смеси сортов винограда, которое может иметь насыщенный вкус и аромат, подходящий для стейка.

Важно помнить, что выбор вина — это дело вкуса, и вы можете экспериментировать с разными сортами, чтобы найти то, что вам больше всего нравится. Также стоит учитывать, что цена и качество вина могут сильно варьироваться, поэтому лучше выбирать проверенные варианты или обратиться за советом к сомелье или продавцу в винном магазине.

Также можно передавать модели полную переписку, включающую в себя системный промпт, сообщения пользователя и ответы ассистента:

In [4]:
res = model.run([
    {
        "role": "system",
        "text": "Ты - опытный сомелье, задача которого - консультировать пользователя в вопросах выбора вин."
    },
    {
        "role": "user",
        "text": "Какое вино можно пить со стейком?"
    }
])
printx(res.text)

Для стейка можно выбрать несколько типов вин, которые хорошо подчеркнут вкус мяса. Вот несколько рекомендаций:

1. **Каберне Совиньон** — это насыщенный и крепкий сорт вина с ярко выраженными танинами, который отлично дополнит вкус стейка.
2. **Мерло** — более мягкий и фруктовый вариант, который также хорошо сочетается с мясом.
3. **Шираз (Сира)** — насыщенное и пряное вино, которое может стать отличным дополнением к стейку.
4. **Мальбек** — особенно аргентинские варианты, которые предлагают насыщенный вкус с нотами тёмного шоколада и фруктов.
5. **Темпранильо** — испанское вино с богатым вкусом и умеренной кислотностью, которое хорошо подходит к стейку.

Выбор вина зависит от ваших личных предпочтений и степени прожарки стейка. Если вы предпочитаете более насыщенный вкус, то каберне совиньон или шираз могут быть отличным выбором. Если же вы любите более мягкие и фруктовые вина, то мерло или темпранильо могут стать хорошим вариантом.

## AI Assistant API

Для ведения беседы с моделью с сохранением контекста диалога используем Assistants API. Объект `thread` будет отвечать за сохранение контекста, а `assistant` - за все основные установки, связанные с работой ассистента.

In [ ]:
exp_params = {
    "ttl_days" : 1, 
    "expiration_policy" : "since_last_active"
}

thread = sdk.threads.create(**exp_params)
assistant = sdk.assistants.create(model, **exp_params)

assistant.update(
    instruction="""Ты - опытный сомелье, задача которого - консультировать пользователя в
    вопросах выбора вина."""
)

thread.write("Привет! Какое вино посоветуете?")

run = assistant.run(thread)
result = run.wait()

printx(result.text)

Здравствуйте! Чтобы подобрать наиболее подходящее вино, мне нужно узнать немного больше о ваших предпочтениях. Какие вина вы пробовали и что вам понравилось? Какой стиль вина вы предпочитаете: лёгкое и свежее или более насыщенное и крепкое? Есть ли у вас предпочтения по сорту винограда или стране-производителю? И наконец, какой у вас бюджет на покупку вина?

In [6]:
thread.write("Я буду есть стейк!")

run = assistant.run(thread)
result = run.wait()

printx(result.text)

Отлично! Для стейка я могу порекомендовать несколько вариантов вина:

1. Каберне Совиньон — это насыщенный и структурированный сорт вина с яркими нотами тёмного винограда и чёрной смородины, которые хорошо сочетаются с мясом.
2. Мерло — более мягкий и фруктовый вариант с нотами вишни и сливы, который также хорошо подойдёт к стейку.
3. Шираз (Сира) — это насыщенное и пряное вино с нотами тёмной вишни, ежевики и специй, которое отлично дополнит вкус стейка.
4. Мальбек — особенно из Аргентины, предлагает насыщенные фруктовые ноты и умеренную сложность, что делает его отличным выбором для стейка.
5. Темпранильо — испанское вино с нотами красных фруктов и ванили, которое хорошо сбалансировано и подходит для мясных блюд.

Если у вас есть предпочтения по стилю или бюджету, пожалуйста, сообщите мне, и я смогу дать более конкретные рекомендации.

В итоге в переписке `thread` содержится вся история сообщений:

In [7]:
for msg in list(thread)[::-1]:
    printx(f"**{msg.author.role}:** {msg.text}")

**USER:** Привет! Какое вино посоветуете?

**ASSISTANT:** Здравствуйте! Чтобы подобрать вино, которое вам понравится, мне нужно знать несколько деталей:

1. Какой тип вина вы предпочитаете: красное, белое, розовое, игристое или десертное?
2. Какой стиль вина вам интересен: лёгкое и свежее или более насыщенное и крепкое?
3. Есть ли у вас предпочтения по вкусам и ароматам? Например, вы любите фруктовые, ягодные, цветочные или более сложные ноты?
4. В каком ценовом диапазоне вы ищете вино?
5. С какими блюдами вы планируете сочетать вино? Это поможет подобрать наиболее подходящий вариант.

Чем больше информации вы предоставите, тем точнее я смогу порекомендовать вино, которое будет соответствовать вашим предпочтениям.

**USER:** Я буду есть стейк!

**ASSISTANT:** Отлично! Для стейка я могу порекомендовать несколько вариантов красного вина:

1. Каберне Совиньон — это насыщенный и структурированный сорт вина с нотами тёмных ягод, дуба и специй, который прекрасно дополнит вкус мяса.
2. Мерло — более мягкий и фруктовый вариант с нотами вишни, сливы и шоколада, который также хорошо подойдёт к стейку.
3. Шираз (Сира) — насыщенное и пряное вино с нотами тёмного фрукта, специй и дуба, которое может стать отличным дополнением к стейку, приготовленному на гриле.
4. Мальбек — это вино с насыщенным вкусом и ароматом тёмных фруктов, ванили и шоколада, которое хорошо подойдёт к стейку средней прожарки.

Это лишь несколько примеров вин, которые могут хорошо сочетаться со стейком. Если у вас есть предпочтения по стилю или вкусу, пожалуйста, сообщите мне, и я смогу дать более конкретные рекомендации. Также важно учитывать ценовой диапазон, в котором вы ищете вино.

После использования переписку и ассистента можно удалить.

In [8]:
thread.delete()
assistant.delete()

## Добавляем RAG

Предположим, мы хотим добавить нашему боту более качественную способность подбирать блюда к вину и наоборот. Для этого найдём таблицу соответствий вин и блюд:

In [8]:
with open('data/food_wine_table.md', encoding='utf-8') as f:
    food_wine = f.readlines()

food_wine_text = "".join(food_wine)
printx(food_wine_text[:1000])

Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Баклажаны, запеченые с сыром | Красное вино: «среднетелые»* сухие — Гренаш (Гарнача), Санджовезе (Кьянти), Карменер, Менсия, молодые Темпранильо, легкотелое Мерло.
Баранина деликатесная (филе или каре ягненка) | Красное вино: сухие выдержанные вина из винограда Пино Нуар, Менсия, Неббиоло (в том числе элегантные выдержанные Бароло и Барбареско), Гамэ (элегантные бургундские Божоле Виляж).
Баранина пикантная: жареная, гриль, тушеная — со специями | Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России.
Бефстроганов | Белые вина: выдержанные в дубе Шардоне, Пино Гриджо (лучше — из Северной Италии), Вердехо, Вермент

Посмотрим на длину этого текста в токенах:

In [14]:
len(model.tokenize(food_wine_text))

12630

## Загружаем файлы в облако

Чтобы RAG мог осущетвлять поиск по фрагментам файла, нам необходимо построить индекс, а перед этим - загрузить все файлы (в нашем случае - один файл) в облако.

In [15]:
up_file = sdk.files.upload("data/food_wine_table.md", **exp_params)

## Строим индекс

Для индексации файлов можно применять следующие стратегии:
* Поиск по эмбеддингам (векторный поиск) - по всем фрагментам текста вычисляются эмбеддинги, и в процессе поиска осуществляется векторный поиск между эмбеддингом запроса и эмбеддингом фрагмента. Это позволяет осуществлять семантический поиск
* Поиск по ключевым словам
* Гибридный поиск, в котором различными способами объединяются результаты поиска по ключевым словам и вектороного поиска.

Также в процессе создания индекса мы указываем **стратегию чанкования** текстов.

In [16]:
from yandex_cloud_ml_sdk.search_indexes import (
    StaticIndexChunkingStrategy,
    HybridSearchIndexType,
    ReciprocalRankFusionIndexCombinationStrategy,
)

op = sdk.search_indexes.create_deferred(
    [up_file],
    index_type=HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens=1000, chunk_overlap_tokens=100
        ),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
    ),
)
index = op.wait()

## Собираем RAG-ассистента

Теперь собираем собственно ассистента, который будет использовать RAG. Для этого определяем **инструмент** (tool) для поиска в нашем индексе, и указываем его при создании ассистента. Также важно задать хорошую инструкцию для ассистента (системный промпт): 

In [17]:
search_tool = sdk.tools.search_index(index)

assistant = sdk.assistants.create(model, tools=[search_tool], **exp_params)
thread = sdk.threads.create(**exp_params)

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если что-то непонятно, то лучше уточни информацию
у пользователя.
"""

_ = assistant.update(instruction=instruction)

In [18]:
thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
printx(result.text)

К стейку можно порекомендовать следующие вина в зависимости от его вида и степени прожарки:

1. Стейк говяжий мраморный нежный (Филе-миньон):
- лёгкие и элегантные из винограда Пино Нуар;
- Нерелло Маскалезе;
- элегантно сделанное и выдержанное Мерло;
- «округлые», выдержанные варианты из сортов Неббиоло (Барбареско);
- Темпранильо (Рибейра дель Дуэро);
- Санджовезе (Кьянти Ризерва).

2. Стейк говяжий мраморный жирноватый (Рибай и пр.):
- к прожарке Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки;
- к прожарке Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6–8 лет выдержки и выше.

Посмотрим, из каких источников был получен этот ответ:

In [20]:
def print_citations(result):
    for citation in result.citations:
        for source in citation.sources:
            if source.type != "filechunk":
                continue
            print("------------------------")
            printx(source.parts[0])

print_citations(result)

------------------------


д. Как вариант — Игристые вина: (экстра-брют и брют) легкие и элегантные белые и утонченные розовые. Салат Оливье | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Как вариант (если ну очень хочется его есть с вином) — Белые вина: недорогие и выдержанные (бочковые) Шардоне, Вердехо, Шенен Блан, Гарганега (Соаве), полусухие и сладковатые Рислинги, а также — Игристые вина: недорогие, но тельные белые (сухое, полусухое) и аналогичные розовые. Неплохо пойдут и Оранжи. Салат Сельдь под шубой | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Как вариант (если ну очень хочется его есть с вином) — Белые вина: недорогие полусухие и сладковатые Рислинги, Совиньон Бланы, Розовые вина: среднетелые и «тельные»  вина Франции, Италии, Испании, России. Как вариант — белые и розовые молодые и недорогие тельные Игристые вина. Неплохо пойдут и Оранжи. Салат фруктовый | Белые вина: полусладкие и сладкие Мускаты и Гевюрцтраминеры, а также — Игристые вина: итальянские Москато д’Асти. Салат Цезарь с креветками | Белые вина: тихие вина из винограда Шардоне, Пино Гри, Шенен Блан, Кортезе (Гави), Пино Блан, Гренаш Блан, полусухой не очень кислотный Рислинг. Игристые вина: белые (экстра-брюты, брюты и сухие) или очень легкие розовые. Неплохо пойдут легкие Оранжи. Салат Цезарь с курицей | Белые вина: сухое и полусухое из винограда Шардоне, Вердехо, Пино Гри, Шенен Блан. Также подойдут Розовые вина: тихие легкие и элегантные вина Прованса (Франция) или Северной Италии. Неплохо пойдут легкие Оранжи (особенно если курятина жарилась на мангале). Салат Цезарь с лососем | Белые вина: сухое выдержанное Шардоне, Альбариньо, Пино Гри (в стиле Альто-Адидже). Полусухие белые вина со средней интенсивностью аромата и вкуса — Семильон, Рислинг, Вердехо. Розовые вина: тихие легкие и элегантные вина Прованса (Франция) или Северной Италии. Салаты азиатские пикантные | Белые вина: сухие и полусухие из винограда Рислинг, Мускат, Гевюрцтраминер. Розовые вина: среднетелые и «тельные»  вина Франции, Италии, Испании, России. Как вариант — розовые молодые и недорогие тельные Игристые вина. К салатам с прикопченым или просто пикантным мясом — можно выбрать легкие Красные вина: Нино Нуары, Нерелло Маскалезе, выдержанное Неббиоло. Неплохо пойдут и Оранжи. Сало соленое или прикопченое | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Свинина в кисло-сладком соусе | Белые вина: Рислинг, сухой Гевюрцтраминер или аналогичный Мускат
Свинина постная, медальоны | Белые вина: тихое сухое выдержанное Шардоне, Альбариньо, Пино Гри (в стиле Альто-Адидже). Полусухие белые вина со средней интенсивностью аромата и вкуса — Семильон, Рислинг, Вердехо. Розовые вина: тихие среднетелые и «тельные» вина Франции, Италии, Испании, России. Свинина стейк жареный с луком | Красные вина: сухие и полусухие вина из винограда Гарнача (Гренаш), Мерло, Карменер, Менсия, «тельные» Пино Нуары (со всего света), российский Красностоп, Гамэ (Божоле Виляж). Сельдь с лучком (закуска) | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Для глобальных приверженцев вина — тельный, сладковатый и мощновкусный Рислинг. Солянка (первое блюдо) | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Спагетти Карбонара | Белые вина: сухие выдержанные вина из Шардоне и Вердехо. Розовые вина: среднетелые и «тельные» вина Франции, Италии, Испании, России.

------------------------


Солянка (первое блюдо) | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Спагетти Карбонара | Белые вина: сухие выдержанные вина из Шардоне и Вердехо. Розовые вина: среднетелые и «тельные» вина Франции, Италии, Испании, России. Красные вина:  выдержанное «округлое» Неббило, элегантные Пино Нуары, Нерелло Маскалезе. Спагетти Болоньезе | Красные вина: сухие из винограда Санджовезе, Барбера, Неро д’Авола, Негроамаро, молодые Неббило, а также международные — Каберне Совиньон, молодоватое испанское Темпранильо. Спаржа обжаренная | Белые вина: тихое сухое из сорта Шардоне, Грюнер Вельтлинер, Мюллер Тургау, Совиньон Блан легкие и не очень кислотный Рислинг, Пино Гри. Как вариант — Игристые вина: «тельное» и капельку сладковатое (брют, сухое, полусухое) белое. Стейк говяжий мраморный нежный (Филе-миньон) | Красные вина: легкие и элегантные из винограда Пино Нуар, Нерелло Маскалезе, элегантно сделанное и выдержанное Мерло. Также подойдут «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва). Стейк говяжий мраморный жирноватый (Рибай и пр.) | Красные вина: к прожарке Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки. К прожарке Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6-8 лет выдержки и выше. Суши, сашими | см. Роллы
Сыры козие молодые | Белые вина: тихие сухие — Совиньон Блан (желательно старосветского стиля — Пюи Фуме, Сансер), Мелон де Бургонь (французские Мюскаде), Шардоне (стиль Шабли). Как вариант — Игристые вина:  розовые (экстра-брют, брют, сухое). Сыры козие выдержанные | Белые вина: сладковатый Рислинг, выдержанное «сливочное» Шардоне, классические Гевюрцтраминер или Мускат. Также — выдержанные бордосские Семийоны и «бочковой» Совиньон Блан (Сотерн). Сыры мягкие с белой плесенью (Бри, Камамбер) | Игристые вина: белые (брют-натюр, экстра-брют, брют), розовые элегантные (экстра-брют, брют). Также — Белые вина: тихое сухое — легкие и элегантные Шардоне, Пино Гри, Виура… Как вариант — утонченное тихое Розовые вина: элегантные вина Прованса (Франция). Сыры мягкие с голубой плесенью (Дор Блю и т. д.) | Белые вина: полусладкие и сладкие вина из Гевюрцтраминера и Муската, сладковатые Рислинги (Ауслезе, Бенернауслезе, Айсвайн), плотноватые и маслянистые Семийоны, выдержанные бочковые Совиньон Бланы, сладкие белые портвейны и десетные вина. Сыры полутвердые сладковатые (Масдам, Гауда, Эдам и т. д.) | Красные вина: тихие сухие вина — ассамбляжи Бордо, «ронские» ассамбляжи Гренаш+Сира+Мурведр, Каберне Совиньоны, вина из Санджовезе, Неро-д-Авола и Темпранильо. Как вариант — «тельные Пино Нуары», Гамэ (Божоле Виляж).

------------------------


Белые вина: сухое выдержанное Шардоне, Альбариньо, Пино Гри (в стиле Альто-Адидже). Полусухие белые вина со средней интенсивностью аромата и вкуса — Семильон, Рислинг, Вердехо. Розовые вина: тихие легкие и элегантные вина Прованса (Франция) или Северной Италии. Рыба красная (семга, форель) пикантная — гриль | Белые вина: тихие — сухие и полусухие яркие Рислинги, выдержанные Шардоне и Вионье, Совиньон Блан из Нового Света. Розовые вина: тихие среднетелые сухие вина Франции, Италии, Испании, России. Как вариант — Красные вина: легкие вина из Пино Нуара, Нерелло Маскалезе и даже Гамэ (выдержанные Божоле Виляж). Подойдут также Оранжи с весомой вкусо-ароматикой. Рыба — стерлядь, осетр (деликатесные, прикопченые) | Русская классика — Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Для любителей вина — настоятельно рекомендую Белые вина: тихие Шардоне (бургундского стиля), сухие и полусухие слегка сладковатые Рислинги невысокой кислотности, Пино Гри (из Альто Адидже), среднетелые и среднекислотные Альбариньо и Вердехо. Как вариант — Игристые вина (брюты и сухие — белые и легчайшие розовые). Рыба Тунец | В зависимости от типа приготовления (от самого нежного к зажаренному и пикантному) — Белые вина: тихие и элегантные выдержанные Шардоне и Вионье, сухие и полусухие Рислинги, Совиньон Блан из Нового Света. Розовые вина: тихие среднетелые сухие вина Франции, Италии, Испании, России. Как вариант к пикантному — Красные вина: легкие вина из Пино Нуара, Нерелло Маскалезе, Гриньолино (Италия), «округленькие» выдержанные Неббиоло и даже Гамэ (выдержанные Божоле Виляж). Подойдут также Оранжи. Салат Греческий | Белые вино: легкие, яркие сухие вина в первую очередь — из «родного» винограда Ассиртико, но отлично также подойдут Мюллер Тургау, Грюнер Вельтллинер, Вердехо, Шенен Блан, Совиньон Блан, Мелон де Бургонь (французские Мюскаде), вина из португальского региона Винью Верде. Как вариант — Игристые вино: белые и очень легкие розовые (брют-натур, экстра-брют, брют, сухое). Салат Еврейский | Белые вино: сухие выдержанные — Шардоне, Альбариньо, Пино Гри (в стиле Альто-Адидже), Семийон, полусухой сладковатый Рислинг, Вердехо. Розовые вина: тихие сухие и полусухие среднетелые вина Франции, Италии, Испании, России. Как вариант — белые и розовые молодые и недорогие тельные Игристые вина. Салат Капрезе | Белые вина: игристые (брют, сухие), тихие вина — с ярким и свежим вкусом: молодые Шардоне, Рислинги, Мюллер Тургау, Грюнер Вельтлинер, Виура, «старосветский» Совиньон Блан, Пино Гри, Фалангина и т. д. Как вариант — Розовые вина: тихие легкие и элегантные вина Прованса (Франция) или Северной Италии, в также попробуйте Игристые вина: легкие и элегантные белые и розовые. Салат Нисуаз | Розовые вина: тихие легкие и элегантные вина Прованса (Франция) или Северной Италии, в также попробуйте Игристые вина: легкие и элегантные белые и розовые. Салат Овощной микс (помидоры, огурцы, болгарский перец, кочанные салаты, зелень и т. д.) | Белые вина: игристые (брют, сухие), тихие вина — с ярким и свежим вкусом: молодые Шардоне, Рислинги, Мюллер Тургау, Грюнер Вельтлинер, Виура, Совиньон Блан, Гренаш Блан, Кортезе (Гави) и т. д. Как вариант — Игристые вина: (экстра-брют и брют) легкие и элегантные белые и утонченные розовые. Салат Оливье | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка.

------------------------


Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Баклажаны, запеченые с сыром | Красное вино: «среднетелые»* сухие — Гренаш (Гарнача), Санджовезе (Кьянти), Карменер, Менсия, молодые Темпранильо, легкотелое Мерло. Баранина деликатесная (филе или каре ягненка) | Красное вино: сухие выдержанные вина из винограда Пино Нуар, Менсия, Неббиоло (в том числе элегантные выдержанные Бароло и Барбареско), Гамэ (элегантные бургундские Божоле Виляж). Баранина пикантная: жареная, гриль, тушеная — со специями | Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России. Бефстроганов | Белые вина: выдержанные в дубе Шардоне, Пино Гриджо (лучше — из Северной Италии), Вердехо, Верментино. Не очень мощные по вкусе и ароматике и кислотности (и недорогие) Рислинги. Блинчики с мясом говядина | Крепкие напитки: Водка, Полугар, Хреновуха. Также — Красные вина: итальянские из Санджовезе (Кьянти), «бордосский» бленд (), «ронский» ассамбляж Гренаш+Сира+Мурведр. Блинчики с мясом курятина | Крепкие напитки: Водка, Полугар, Хреновуха. Также можно пить Белые вина: из винограда Шардоне, Вердехо, Верментино, Треббьяно, Пино Гри, сухие Хересы. Блины с красной рыбой или икрой | Крепкие напитки: Водка, Полугар, Хреновуха. Белые вина: из винограда Рислинг, Альбариньо, сухой Мускат или Гевюрцтраминер, сухой Херес. Борщ | Крепкие напитки: Самогон (Полугар, Хлебное вино), Водка, Хреновуха. Говядина жареная, тушеная (кроме мраморных стейков) | Красные вина: сухие вина из винограда Санджовезе (Кьянти) и Неро д’Авола, испанские Темпранильо, «тельные» Мерло, «тельные» Пино Нуары (из Нового Света, Австрии, Германии), вина из Бордо, вина из Роны (Гренаш+Сира+Мурведр), Саперави (как из Грузии, так и из России), Арени (Армения). Грибы лесные, благородные — жареные, тушеные | К средневкусным — Белые вина: сухие и полусухое выдержанные Шардоне, Вионье, сухой Гевюрцтраминер. К более пикантным — Красные вина: сухие вина из винограда Пино Нуар, Нерелло Москалезе, выдержанное и бархатистое Неббиоло. Грибы деликатесные (шампиньоны, лисички и пр.) | Белые вина: сухие и полусухое выдержанные Шардоне, Вионье, сухой Гевюрцтраминер (к лисичкам, шампиньонам в сливках). Розовые вина: тихие и Игристые (особенно из Пино Нуара). Жульен с грибами | Красные вина: сухие из винограда Пино Нуар, Менсия, Нерелло Москалезе, выдержанное и бархатистое Неббиоло. Розовые вина: среднетелые и полнотелые из винограда Гренаш, Сенсо, Темпранильо, Бобаль, Сира, или «тельные» розовые Игристые. Заливное — белая рыба | Крепкие напитки: Водка, Полугар, Хреновуха. Как вариант — Белые вина: Альбариньо, Вердехо, Совиньон Блан, Мюллер Тургау, Рислинг. Икра красная на крекерах, белом хлебе | Белые вина: тихие, капельку сладковатые и выдержанные Пино Гриджо, Пино Блан, Шенен Блан, Вионье, Семийон, Гарганега (Соаве). Утонченные вина из Шардоне (в стиле Шабли).

------------------------


Совиньон Блан (старосветского стиля — типа Сансера)
Куриная грудка — гриль в специях | Белые вина: сухие Гевюрцтраминеры и Мускаты, выдержанные в дубе Шардоне, полнотелые Рислинги. Розовые вина: широкий диапазон вин из Прованса (Франция), Италии, Испании и т. д. Зависит от пикантности (степени зажарки и специй). Курица пикантная — Азиатская кухня (цитрусовый или кисло-сладкий соус) | Белые вина: яркий и «тельный» (возможно полусухой) Рислинг, сухой Гевюрцтраминер или Мускат, Торронтес. Вина Чаколи из Страны Басков. Курица-гриль | Розовые вина: полнотелые сухие и полусухие вина из Франции, Италии, Испании, России и т.д. Красные вина: не очень дорогие вина из: Пино Нуара, Менсии, Гарначи (Гренаш), элегантные вина из Мерло и Каберне Фран. Полусладкое грузинское Саперави (Киндзмараули). Российские вина из Красностопа. Лагман | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Лазанья мясная | Красные вина: сухие вина из Санджовезе (Кьянти), Гарначи (Гренаш), невыдержанные из Темпранильо, молодые вина Бордо с доминантой Мерло. Российские вина из Мерло и Каберне Франа. Лазанья овощная | Розовые вина: полнотелые сухие (и даже — полусухие) вина из Прованса (Франция), Италии, Испании, России. Красные вина: легкие и элегантные — из Нерелло Маскалезе, Пино Нуара (берите не очень дорогие вина Нового Света, немецкие или австрийские), если найдете — элегантно сделанное Мерло. Лобстер в сливочном (сырном) соусе | Белые вина: сухое элегантное вино с невысокой кислотностью из винограда Шардоне (Шабли, Мерсо), Кортезе (Гави), Вердехо, Гарганега (Соаве), Пино Гри, Пино Блан, Грюнер Вельлинер. Как вариант — миллезимные белые Игристые. При более пикантном, сырном соусе — Розовые вина: «легкотелые» варианты из Прованса и Италии. Люля-кебаб классика (баранина) | Красные вина: не очень дорогие вина из сортов — Санджовезе (Кьянти), Темпранильо (в том числе Crianza), Каберне Совиньон (весь мир), Сира (Шираз), Монастрель (Мурведр). Также хороши — молодые, но довольно «тельные» вина из Бордо, Арени (Армения), весомые красные Кипра и Греции (Маратефтико, Ксиномавро), Примитиво (Занфандель), российские вина из Саперави, южноафриканский Пинотаж. Медальоны из телятины | Красные вина: легкие и элегантные из винограда Пино Нуар, Нерелло Маскалезе, элегантно сделанное и выдержанное Мерло. Также подойдут «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва). Медальоны из свинины | Белые вина: сухое и полусухое выдержанное из винограда Шардоне, Вердехо, Пино Гри, Шенен Блан. Также подойдут Розовые вина: тихие «среднетелые» из Франции, Италии, Испании, России. Неплохо пойдут Оранжи. Морепродукты жареные (гриль) — устрицы, гребешки, лобстеры, осьминоги, мидии и пр. | Белые вина: сухое и полусухое, «тельное» с хорошей кислотностью из сортов: Шардоне, Вердехо, Верментино, Рислинг, Альбариньо, Гарганега (Соаве). Розовые вина: сухие или полусухие вина Франции, Италии, Испании, Германии, России. Нарезка мясная деликатесная варено-копченая (в т.ч. колбаса) | Белые вина: тихие сухие (полусухие) довольно «яркие» вина со средней кислотностью из винограда сортов Шардоне, Вердехо, Верментино, Рислинг, Пино Гри. Как вариант — Игристое вино: молодое кислотноватое белое или розовое — брют или экстра-брют. Нарезка мясная деликатесная сыро-копченая-вяленая (в т.ч.

In [17]:
thread.delete()

## Ручное чанкование таблицы

В таблице важную роль играет **заголовок** - он задаёт семантику столбцов. В предыдущем примере модель отвечала неплохо, но в случае, если есть много разных RAG-фрагменов, наличие семантики может оказаться важным. Поэтому лучше разбивать табличку на фрагменты так, чтобы заголовок таблицы сохранялся в каждом фрагменте.

Для этого запомним отдельно заголовок таблицы:

In [21]:
header = food_wine[:2]
header

['Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду\n',
 '--------|--------\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [22]:
chunk_size = 600 * 3  # approx 600 tokens * 2 char/token

s = header.copy()
uploaded_foodwine = []
for x in food_wine[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        id = sdk.files.upload_bytes(
            "".join(s).encode(),
            mime_type="text/markdown",
            **exp_params
        )
        #printx("".join(s))
        uploaded_foodwine.append(id)
        s = header.copy()
print(f"Uploaded {len(uploaded_foodwine)} table chunks")

Uploaded 22 table chunks


Теперь создадим индекс на основе этих фрагментов, и ассистента на основе индекса:

In [23]:
op = sdk.search_indexes.create_deferred(
    uploaded_foodwine,
    index_type=HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens=1000, chunk_overlap_tokens=100
        ),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
    ),
)
index = op.wait()

search_tool = sdk.tools.search_index(index)

assistant = sdk.assistants.create(model, tools=[search_tool], **exp_params)

Посмотрим, как такой ассистент работает:

In [24]:
thread = sdk.threads.create(**exp_params)

thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
printx(result.text)
print_citations(result)

Выбор вина к стейку зависит от его вида и степени прожарки.

* **Стейк из нежной мраморной говядины (филе-миньон)**: подойдут лёгкие и элегантные красные вина из винограда Пино Нуар, Нерелло Маскалезе, а также выдержанное и элегантное Мерло. Можно рассмотреть «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).
* **Стейк из мраморной говядины с более высоким содержанием жира (Рибай и подобные)**: для прожарки Rare рекомендуются выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки. Для прожарки Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6–8 лет выдержки и выше.

------------------------


Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Солянка (первое блюдо) | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Спагетти Карбонара | Белые вина: сухие выдержанные вина из Шардоне и Вердехо. Розовые вина: среднетелые и «тельные» вина Франции, Италии, Испании, России. Красные вина:  выдержанное «округлое» Неббило, элегантные Пино Нуары, Нерелло Маскалезе. Спагетти Болоньезе | Красные вина: сухие из винограда Санджовезе, Барбера, Неро д’Авола, Негроамаро, молодые Неббило, а также международные — Каберне Совиньон, молодоватое испанское Темпранильо. Спаржа обжаренная | Белые вина: тихое сухое из сорта Шардоне, Грюнер Вельтлинер, Мюллер Тургау, Совиньон Блан легкие и не очень кислотный Рислинг, Пино Гри. Как вариант — Игристые вина: «тельное» и капельку сладковатое (брют, сухое, полусухое) белое. Стейк говяжий мраморный нежный (Филе-миньон) | Красные вина: легкие и элегантные из винограда Пино Нуар, Нерелло Маскалезе, элегантно сделанное и выдержанное Мерло. Также подойдут «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва). Стейк говяжий мраморный жирноватый (Рибай и пр.) | Красные вина: к прожарке Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки. К прожарке Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6-8 лет выдержки и выше. Суши, сашими | см. Роллы

------------------------


Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Салат Цезарь с курицей | Белые вина: сухое и полусухое из винограда Шардоне, Вердехо, Пино Гри, Шенен Блан. Также подойдут Розовые вина: тихие легкие и элегантные вина Прованса (Франция) или Северной Италии. Неплохо пойдут легкие Оранжи (особенно если курятина жарилась на мангале). Салат Цезарь с лососем | Белые вина: сухое выдержанное Шардоне, Альбариньо, Пино Гри (в стиле Альто-Адидже). Полусухие белые вина со средней интенсивностью аромата и вкуса — Семильон, Рислинг, Вердехо. Розовые вина: тихие легкие и элегантные вина Прованса (Франция) или Северной Италии. Салаты азиатские пикантные | Белые вина: сухие и полусухие из винограда Рислинг, Мускат, Гевюрцтраминер. Розовые вина: среднетелые и «тельные»  вина Франции, Италии, Испании, России. Как вариант — розовые молодые и недорогие тельные Игристые вина. К салатам с прикопченым или просто пикантным мясом — можно выбрать легкие Красные вина: Нино Нуары, Нерелло Маскалезе, выдержанное Неббиоло. Неплохо пойдут и Оранжи. Сало соленое или прикопченое | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Свинина в кисло-сладком соусе | Белые вина: Рислинг, сухой Гевюрцтраминер или аналогичный Мускат
Свинина постная, медальоны | Белые вина: тихое сухое выдержанное Шардоне, Альбариньо, Пино Гри (в стиле Альто-Адидже). Полусухие белые вина со средней интенсивностью аромата и вкуса — Семильон, Рислинг, Вердехо. Розовые вина: тихие среднетелые и «тельные» вина Франции, Италии, Испании, России. Свинина стейк жареный с луком | Красные вина: сухие и полусухие вина из винограда Гарнача (Гренаш), Мерло, Карменер, Менсия, «тельные» Пино Нуары (со всего света), российский Красностоп, Гамэ (Божоле Виляж). Сельдь с лучком (закуска) | Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Для глобальных приверженцев вина — тельный, сладковатый и мощновкусный Рислинг.

...

Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Рыба — стерлядь, осетр (деликатесные, прикопченые) | Русская классика — Крепкие напитки: Самогоны (Полугар, Хлебное вино), Водка, Хреновуха, Перцовка. Для любителей вина — настоятельно рекомендую Белые вина: тихие Шардоне (бургундского стиля), сухие и полусухие слегка сладковатые Рислинги невысокой кислотности, Пино Гри (из Альто Адидже), среднетелые и среднекислотные Альбариньо и Вердехо. Как вариант — Игристые вина (брюты и сухие — белые и легчайшие розовые). Рыба Тунец | В зависимости от типа приготовления (от самого нежного к зажаренному и пикантному) — Белые вина: тихие и элегантные выдержанные Шардоне и Вионье, сухие и полусухие Рислинги, Совиньон Блан из Нового Света. Розовые вина: тихие среднетелые сухие вина Франции, Италии, Испании, России. Как вариант к пикантному — Красные вина: легкие вина из Пино Нуара, Нерелло Маскалезе, Гриньолино (Италия), «округленькие» выдержанные Неббиоло и даже Гамэ (выдержанные Божоле Виляж). Подойдут также Оранжи. Салат Греческий | Белые вино: легкие, яркие сухие вина в первую очередь — из «родного» винограда Ассиртико, но отлично также подойдут Мюллер Тургау, Грюнер Вельтллинер, Вердехо, Шенен Блан, Совиньон Блан, Мелон де Бургонь (французские Мюскаде), вина из португальского региона Винью Верде. Как вариант — Игристые вино: белые и очень легкие розовые (брют-натур, экстра-брют, брют, сухое). Салат Еврейский | Белые вино: сухие выдержанные — Шардоне, Альбариньо, Пино Гри (в стиле Альто-Адидже), Семийон, полусухой сладковатый Рислинг, Вердехо. Розовые вина: тихие сухие и полусухие среднетелые вина Франции, Италии, Испании, России. Как вариант — белые и розовые молодые и недорогие тельные Игристые вина. Салат Капрезе | Белые вина: игристые (брют, сухие), тихие вина — с ярким и свежим вкусом: молодые Шардоне, Рислинги, Мюллер Тургау, Грюнер Вельтлинер, Виура, «старосветский» Совиньон Блан, Пино Гри, Фалангина и т. д. Как вариант — Розовые вина: тихие легкие и элегантные вина Прованса (Франция) или Северной Италии, в также попробуйте Игристые вина: легкие и элегантные белые и розовые.

------------------------


Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Грибы лесные, благородные — жареные, тушеные | К средневкусным — Белые вина: сухие и полусухое выдержанные Шардоне, Вионье, сухой Гевюрцтраминер. К более пикантным — Красные вина: сухие вина из винограда Пино Нуар, Нерелло Москалезе, выдержанное и бархатистое Неббиоло. Грибы деликатесные (шампиньоны, лисички и пр.) | Белые вина: сухие и полусухое выдержанные Шардоне, Вионье, сухой Гевюрцтраминер (к лисичкам, шампиньонам в сливках). Розовые вина: тихие и Игристые (особенно из Пино Нуара). Жульен с грибами | Красные вина: сухие из винограда Пино Нуар, Менсия, Нерелло Москалезе, выдержанное и бархатистое Неббиоло. Розовые вина: среднетелые и полнотелые из винограда Гренаш, Сенсо, Темпранильо, Бобаль, Сира, или «тельные» розовые Игристые. Заливное — белая рыба | Крепкие напитки: Водка, Полугар, Хреновуха. Как вариант — Белые вина: Альбариньо, Вердехо, Совиньон Блан, Мюллер Тургау, Рислинг. Икра красная на крекерах, белом хлебе | Белые вина: тихие, капельку сладковатые и выдержанные Пино Гриджо, Пино Блан, Шенен Блан, Вионье, Семийон, Гарганега (Соаве). Утонченные вина из Шардоне (в стиле Шабли). Икра черная на крекерах, белом хлебе | Несмотря на распространенные советы, сочетать черную икру с винами тихими и особенно — игристыми, не рекомендую. Однако, если очень хочется потреблять черную икру со спиртным, то мой вам совет — сначала, глоток Водки или любого иного невыдержанного в дубе Дистиллята, а уже потом — закусить чистой икрой или бутербродом с ней. Индейка (стейки, котлеты, шашлычки) | Белые вина: сухое и полусухое выдержанное из винограда сортов Шардоне, Вердехо, Верментино, Семийон, Пино Гри, Шенен Блан. Также подойдут «среднетелые» Розовые вина или Оранжи. Каре ягненка | Красные вина: «среднетелые» вина со сбалансированной кислотностью — хорошо выдержанные «супертосканские» вина, Брунелло ди Монтальчино, Кьянти Ризерва, округлые и сочные вина из Неро д’Авола, не слишком тельные вина из Сиры (Шираза), резервные вина из Риохи, добротно сделанные Гарнача и Барбера, «тельные» Пино Нуары (из Нового Света, Австрии, Германии), созревшие и отлично сбалансированные вина Бордо и Пьемонта (в том числе Бароло и Барбареско), выдержанное Арени (Армения).

------------------------


Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Баклажаны, запеченые с сыром | Красное вино: «среднетелые»* сухие — Гренаш (Гарнача), Санджовезе (Кьянти), Карменер, Менсия, молодые Темпранильо, легкотелое Мерло. Баранина деликатесная (филе или каре ягненка) | Красное вино: сухие выдержанные вина из винограда Пино Нуар, Менсия, Неббиоло (в том числе элегантные выдержанные Бароло и Барбареско), Гамэ (элегантные бургундские Божоле Виляж). Баранина пикантная: жареная, гриль, тушеная — со специями | Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России. Бефстроганов | Белые вина: выдержанные в дубе Шардоне, Пино Гриджо (лучше — из Северной Италии), Вердехо, Верментино. Не очень мощные по вкусе и ароматике и кислотности (и недорогие) Рислинги. Блинчики с мясом говядина | Крепкие напитки: Водка, Полугар, Хреновуха. Также — Красные вина: итальянские из Санджовезе (Кьянти), «бордосский» бленд (), «ронский» ассамбляж Гренаш+Сира+Мурведр. Блинчики с мясом курятина | Крепкие напитки: Водка, Полугар, Хреновуха. Также можно пить Белые вина: из винограда Шардоне, Вердехо, Верментино, Треббьяно, Пино Гри, сухие Хересы. Блины с красной рыбой или икрой | Крепкие напитки: Водка, Полугар, Хреновуха. Белые вина: из винограда Рислинг, Альбариньо, сухой Мускат или Гевюрцтраминер, сухой Херес. Борщ | Крепкие напитки: Самогон (Полугар, Хлебное вино), Водка, Хреновуха. Говядина жареная, тушеная (кроме мраморных стейков) | Красные вина: сухие вина из винограда Санджовезе (Кьянти) и Неро д’Авола, испанские Темпранильо, «тельные» Мерло, «тельные» Пино Нуары (из Нового Света, Австрии, Германии), вина из Бордо, вина из Роны (Гренаш+Сира+Мурведр), Саперави (как из Грузии, так и из России), Арени (Армения).

Видим, что ответ почти не изменился, зато чанки, на которые смотрела модель, выглядят намного более аккуратными, и содержат в себе заголовок таблицы.

In [24]:
thread.delete()
assistant.delete()

## Удаляем лишнее

Если вы хотите на всякий случай удалить созданные в процессе ресурсы, то можете использовать код ниже - он удалит все переписки, файлы, индексы и ассистенты. 

**ВНИМАНИЕ**: Не выполняйте этот код, если у вас есть другие проекты с ассистентами в облаке (точнее - в каталоге)!

In [ ]:
for thread in sdk.threads.list():
    print(f" + deleting thread id={thread.id}",end="")
    try:
        thread.delete()
    except:
        print(" ! Error",end="")
    print()
        
for assistant in sdk.assistants.list():
    print(f" + deleting assistant id={assistant.id}")
    assistant.delete()

In [97]:
from tqdm.auto import tqdm

for index in sdk.search_indexes.list():
    print(f" + deleting index id={index.id}")
    index.delete()
    
print(" + Deleting files")
for file in tqdm(sdk.files.list()):
    # print(f" + deleting file id={file.id}")
    file.delete()

 + Deleting files


0it [00:00, ?it/s]
